<a href="https://colab.research.google.com/github/urmil24/AUTOMATED-REPORT-GENERATION/blob/main/AUTOMATED_REPORT_GENERATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# ================= Install Required Libraries ==================
!pip install kagglehub fpdf matplotlib --quiet

# ================= Import Required Libraries ===================
import pandas as pd
import matplotlib.pyplot as plt
from fpdf import FPDF
import kagglehub
from kagglehub import KaggleDatasetAdapter

# ================= Load Kaggle Dataset =========================
df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "jacksoncrow/stock-market-dataset",
    "stocks/AAPL.csv"
)

# ================= Preprocess the Data =========================
df['Date'] = pd.to_datetime(df['Date'])        # Convert Date column to datetime
df.sort_values('Date', inplace=True)           # Sort the data by Date

# ================= Perform Basic Analysis ======================
latest_data = df.tail(1)
average_close = df['Close'].mean()
max_close = df['Close'].max()
min_close = df['Close'].min()
total_volume = df['Volume'].sum()

# ================= Moving Average Calculation ==================
df['MA20'] = df['Close'].rolling(window=20).mean()
df['MA50'] = df['Close'].rolling(window=50).mean()

# ================= Save Line Chart of Price and MAs ============
plt.figure(figsize=(10, 5))
plt.plot(df['Date'], df['Close'], label='Close Price', color='blue')
plt.plot(df['Date'], df['MA20'], label='20-day MA', color='orange')
plt.plot(df['Date'], df['MA50'], label='50-day MA', color='green')
plt.title("AAPL Stock Prices with Moving Averages")
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend()
plt.tight_layout()
plt.savefig("aapl_price_chart.png")
plt.close()

# ================= Save Bar Chart of Volume =====================
plt.figure(figsize=(10, 4))
plt.bar(df.tail(10)['Date'].dt.strftime('%Y-%m-%d'), df.tail(10)['Volume'], color='purple')
plt.xticks(rotation=45)
plt.title("AAPL - Volume (Last 10 Days)")
plt.xlabel("Date")
plt.ylabel("Volume")
plt.tight_layout()
plt.savefig("aapl_volume_chart.png")
plt.close()

# ==================== PDF Report Class ==========================
class PDF(FPDF):
    def header(self):
        self.set_font("Arial", "B", 14)
        self.cell(0, 10, "AAPL Stock Market Report", ln=True, align="C")
        self.ln(5)

    def footer(self):
        self.set_y(-15)
        self.set_font("Arial", "I", 10)
        self.cell(0, 10, f"Page {self.page_no()}", align="C")

    def add_stats(self, avg, max_, min_, volume, latest_row):
        self.set_font("Arial", "", 12)
        self.cell(0, 10, f"Average Close Price: ${avg:.2f}", ln=True)
        self.cell(0, 10, f"Highest Close Price: ${max_:.2f}", ln=True)
        self.cell(0, 10, f"Lowest Close Price: ${min_:.2f}", ln=True)
        self.cell(0, 10, f"Total Volume Traded: {int(volume):,}", ln=True)
        date_str = latest_row['Date'].dt.date.values[0]
        price_str = latest_row['Close'].values[0]
        self.cell(0, 10, f"Most Recent Close: ${price_str:.2f} on {date_str}", ln=True)
        self.ln(10)

    def add_table(self, data):
        self.set_font("Arial", "B", 12)
        self.cell(40, 10, "Date", border=1)
        self.cell(40, 10, "Open", border=1)
        self.cell(40, 10, "Close", border=1)
        self.cell(40, 10, "Volume", border=1)
        self.ln()
        self.set_font("Arial", "", 11)
        for _, row in data.iterrows():
            self.cell(40, 10, str(row["Date"].date()), border=1)
            self.cell(40, 10, f"{row['Open']:.2f}", border=1)
            self.cell(40, 10, f"{row['Close']:.2f}", border=1)
            self.cell(40, 10, f"{int(row['Volume']):,}", border=1)
            self.ln()

    def add_image(self, path, title):
        self.set_font("Arial", "B", 12)
        self.cell(0, 10, title, ln=True)
        self.image(path, w=180)
        self.ln(10)

# =================== Generate PDF Report ========================
pdf = PDF()
pdf.add_page()
pdf.add_stats(average_close, max_close, min_close, total_volume, latest_data)
pdf.add_image("aapl_price_chart.png", "Stock Price with Moving Averages")
pdf.add_image("aapl_volume_chart.png", "Volume (Last 10 Days)")
pdf.add_table(df.tail(10))  # Last 10 records

# =================== Save PDF File ==============================
pdf.output("aapl_stock_report.pdf")

print("PDF report generated successfully: 'aapl_stock_report.pdf'")


<ipython-input-4-1023751404>:12: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


PDF report generated successfully: 'aapl_stock_report.pdf'
